In [1]:
from Bio import SeqIO
from Bio.Seq import Seq
import numpy as np
import pandas as pd
import random

In [2]:
meta=[]
sequence=[]

In [3]:
seq = ('LongInternalExon_SNM.txt')

In [4]:
for seq_record in SeqIO.parse(seq, "fasta"):
    meta.append(str(seq_record.id))
    sequence.append(str(seq_record.seq))

In [5]:
Fasta = pd.DataFrame(data ={'Meta':meta,'Sequence':sequence})

In [6]:
Data = pd.read_csv("LongInternalExon_SNM.csv")

In [7]:
Result = pd.merge(Data, Fasta, left_index=True, right_index=True, how='outer')

In [8]:
Result.drop(["Meta"], axis=1, inplace=True)

In [15]:
Result

,name,chrom,strand,txStart,txEnd,cdsStart,cdsEnd,exonStart,exonEnd,ExonLength,CumSum,m6AStart,m6AEnd,Probabilty,Label,Sequence,GeneLength
0,Satb1,chr17,-,51736186,51832672,51739897,51809253,51736186,51740413,4227,6243,51739922,51739923,0.936465,1,TCATGTTTTAAACATTAATTCCAAACTATTTTTATTTAGCATATGG...,96486
1,Setd5,chr6,+,113077364,113153435,113105050,113151470,113150861,113153435,2574,6755,113151460,113151461,0.930645,1,AGTGGCGTTTCCGCTCGGGCAGCGGGCTGAGTGAgctgccgccgct...,76071
2,Jag1,chr2,-,137081455,137116644,137083023,137116257,137081455,137083481,2026,5612,137083096,137083097,0.930130,1,CATGTTTTCATACaaaaaatttaataaatattacttttcaaaattt...,35189
3,Six4,chr12,-,73099608,73113425,73103441,73113184,73108660,73109349,689,1772,73109072,73109073,0.928997,1,TATGTATAAATATAAAGTTTATTTAGCATAGTGTTTAGAAAAATAA...,13817
4,Zfpm2,chr15,+,40655041,41104592,40655543,41103576,41101084,41104592,3508,4974,41103517,41103518,0.920117,1,GCGCGGCGCGGCCCGGAGCGGCAGCGGCGGCGCCTGAGTCTCTGCC...,449551
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
888,Mrpl57,chr14,+,57826238,57828745,57826516,57826825,57826508,57828745,2237,2321,57826802,57826803,0.403253,1,GAGACCGGAAGCAGCGCGCCGCCGCATCGGTCGGGGGACGCGGAGC...,2507
889,Lrrn3,chr12,-,41451672,41485751,41452192,41454316,41451672,41454671,2999,3180,41452631,41452632,0.403062,1,CCCAGAAATTATTCTTTATTTAGAAACAAAAACAAGACAAAAGCAA...,34079
890,Nxph1,chr6,+,8949675,9249030,8950253,9247846,9247084,9249030,1946,2578,9247347,9247348,0.402985,1,TCTCAGTCTCTTGCTCGCTTGCTTTTGCTATCTCTCTCTGCTGCTG...,299355
891,Golga3,chr5,+,110176700,110226470,110181777,110223079,110187340,110187987,647,1443,110187718,110187719,0.402206,1,AAAATTTTAAGTCCACTTTCTGAAGGATATCTCAATCTGTCCGGAG...,49770


In [9]:
Result["GeneLength"] = Result["txEnd"] - Result["txStart"]

In [16]:
Data = Result

In [17]:
Positive = Data[Data["strand"]=="+"]
Positive = Positive.reset_index(drop = True)
Negative = Data[Data["strand"]=="-"]
Negative = Negative.reset_index(drop = True)

In [18]:
Sequence = Negative["Sequence"].tolist()

In [19]:
New = []
for i in range(len(Sequence)):
    input_sequence = Sequence[i][::-1]
    input_sequence = Seq(input_sequence).complement()
    input_sequence = str(input_sequence)
    New.append(input_sequence)

New = np.array(New)
New = pd.DataFrame({"Sequence":New})

In [20]:
Negative["Sequence"] = 0
Negative["Sequence"] = New["Sequence"]

In [21]:
Data = Positive.append(Negative)
Data = Data.reset_index(drop = True)

In [22]:
Data

,name,chrom,strand,txStart,txEnd,cdsStart,cdsEnd,exonStart,exonEnd,ExonLength,CumSum,m6AStart,m6AEnd,Probabilty,Label,Sequence,GeneLength
0,Setd5,chr6,+,113077364,113153435,113105050,113151470,113150861,113153435,2574,6755,113151460,113151461,0.930645,1,AGTGGCGTTTCCGCTCGGGCAGCGGGCTGAGTGAgctgccgccgct...,76071
1,Zfpm2,chr15,+,40655041,41104592,40655543,41103576,41101084,41104592,3508,4974,41103517,41103518,0.920117,1,GCGCGGCGCGGCCCGGAGCGGCAGCGGCGGCGCCTGAGTCTCTGCC...,449551
2,Fzd4,chr7,+,89404365,89410110,89404686,89408360,89407031,89410110,3079,3685,89407633,89407634,0.908575,1,AGCGCTGGGGCGGTGAGAACAGCGCGGCGTAGAGTGCAGGCGGGCT...,5745
3,Pard6b,chr2,+,168081003,168101203,168081293,168099210,168098383,168101203,2820,3399,168099166,168099167,0.906320,1,AGGGAGGCTGCGGCGGCTGTAGGGCTGTGCTCCGCCCTCTCCTCAc...,20200
4,Rif1,chr2,+,52072836,52122381,52073149,52121203,52109910,52113000,3090,6508,52112643,52112644,0.898967,1,ATCTTGGTCGTGGAGGAGCGGGCTGCACGCGTGAGTAAATAAGCGC...,49545
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
888,Slc22a23,chr13,-,34179157,34345182,34182953,34344797,34179157,34183311,4154,6251,34183035,34183036,0.405162,1,GAAGTCCGTGGAGGATCTCCCAAcccgcggccgccggcggccccAG...,166025
889,Eri2,chr7,-,119784895,119794058,119785209,119793940,119784895,119786544,1649,2499,119785365,119785366,0.404177,1,GACGGCCGCCGCCGCTCTACAAGCGCCCGCTTCCCTTGCCGCGCCG...,9163
890,Wtap,chr17,-,12966798,12992259,12967467,12985945,12966798,12968051,1253,2051,12967501,12967502,0.403611,1,agtccgggcggggcggggccggcggcagggccggcTTTCCTCCCTC...,25461
891,Lrrn3,chr12,-,41451672,41485751,41452192,41454316,41451672,41454671,2999,3180,41452631,41452632,0.403062,1,agagagaCACCATTGCTTGCATGTGTGCATATCTATAGTATGTAAT...,34079


In [23]:
Positive = Data[Data["strand"]=="+"]
Positive = Positive.reset_index(drop = True)
Negative = Data[Data["strand"]=="-"]
Negative = Negative.reset_index(drop = True)

In [24]:
Positive["POS"] = Positive["m6AStart"] - Positive["txStart"]
Negative["POS"] = Negative["txEnd"] - Negative["m6AEnd"]

In [25]:
Data = Positive.append(Negative)
Data = Data.reset_index(drop = True)

In [26]:
for i in range(len(Data)):
    Data.loc[i,("m6ASequence")] = Data.loc[i,("Sequence")][(Data.loc[i,("POS")]-2):(Data.loc[i,("POS")]+3)]

In [27]:
Data

,name,chrom,strand,txStart,txEnd,cdsStart,cdsEnd,exonStart,exonEnd,ExonLength,CumSum,m6AStart,m6AEnd,Probabilty,Label,Sequence,GeneLength,POS,m6ASequence
0,Setd5,chr6,+,113077364,113153435,113105050,113151470,113150861,113153435,2574,6755,113151460,113151461,0.930645,1,AGTGGCGTTTCCGCTCGGGCAGCGGGCTGAGTGAgctgccgccgct...,76071,74096,GGACT
1,Zfpm2,chr15,+,40655041,41104592,40655543,41103576,41101084,41104592,3508,4974,41103517,41103518,0.920117,1,GCGCGGCGCGGCCCGGAGCGGCAGCGGCGGCGCCTGAGTCTCTGCC...,449551,448476,GAACT
2,Fzd4,chr7,+,89404365,89410110,89404686,89408360,89407031,89410110,3079,3685,89407633,89407634,0.908575,1,AGCGCTGGGGCGGTGAGAACAGCGCGGCGTAGAGTGCAGGCGGGCT...,5745,3268,GGACT
3,Pard6b,chr2,+,168081003,168101203,168081293,168099210,168098383,168101203,2820,3399,168099166,168099167,0.906320,1,AGGGAGGCTGCGGCGGCTGTAGGGCTGTGCTCCGCCCTCTCCTCAc...,20200,18163,GGACC
4,Rif1,chr2,+,52072836,52122381,52073149,52121203,52109910,52113000,3090,6508,52112643,52112644,0.898967,1,ATCTTGGTCGTGGAGGAGCGGGCTGCACGCGTGAGTAAATAAGCGC...,49545,39807,GAACT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
888,Slc22a23,chr13,-,34179157,34345182,34182953,34344797,34179157,34183311,4154,6251,34183035,34183036,0.405162,1,GAAGTCCGTGGAGGATCTCCCAAcccgcggccgccggcggccccAG...,166025,162146,GGACT
889,Eri2,chr7,-,119784895,119794058,119785209,119793940,119784895,119786544,1649,2499,119785365,119785366,0.404177,1,GACGGCCGCCGCCGCTCTACAAGCGCCCGCTTCCCTTGCCGCGCCG...,9163,8692,GAACA
890,Wtap,chr17,-,12966798,12992259,12967467,12985945,12966798,12968051,1253,2051,12967501,12967502,0.403611,1,agtccgggcggggcggggccggcggcagggccggcTTTCCTCCCTC...,25461,24757,GGACT
891,Lrrn3,chr12,-,41451672,41485751,41452192,41454316,41451672,41454671,2999,3180,41452631,41452632,0.403062,1,agagagaCACCATTGCTTGCATGTGTGCATATCTATAGTATGTAAT...,34079,33119,AGACT


In [29]:
Data.drop(["m6ASequence"], axis=1, inplace=True)

In [30]:
for i in range(len(Data)):
    Data.loc[i,("Up")] = Data.loc[i,("Sequence")][:(Data.loc[i,("POS")]-250)]
    Data.loc[i,("Mid")] = Data.loc[i,("Sequence")][(Data.loc[i,("POS")]-250):(Data.loc[i,("POS")]+251)]
    Data.loc[i,("Down")] = Data.loc[i,("Sequence")][(Data.loc[i,("POS")]+251):]

In [31]:
Data

,name,chrom,strand,txStart,txEnd,cdsStart,cdsEnd,exonStart,exonEnd,ExonLength,...,m6AStart,m6AEnd,Probabilty,Label,Sequence,GeneLength,POS,Up,Mid,Down
0,Setd5,chr6,+,113077364,113153435,113105050,113151470,113150861,113153435,2574,...,113151460,113151461,0.930645,1,AGTGGCGTTTCCGCTCGGGCAGCGGGCTGAGTGAgctgccgccgct...,76071,74096,AGTGGCGTTTCCGCTCGGGCAGCGGGCTGAGTGAgctgccgccgct...,CCTCAGACTCGGTTTCTCAGTCCAGCACAGGAACTCTGAGTTCCAC...,TTAACAAGCAAATGGTGTTTCGGTTAGTAACGGTTCTAAGTGCAAT...
1,Zfpm2,chr15,+,40655041,41104592,40655543,41103576,41101084,41104592,3508,...,41103517,41103518,0.920117,1,GCGCGGCGCGGCCCGGAGCGGCAGCGGCGGCGCCTGAGTCTCTGCC...,449551,448476,GCGCGGCGCGGCCCGGAGCGGCAGCGGCGGCGCCTGAGTCTCTGCC...,CAAGATGAAAGACCTACGGCCAACCCACAGCAAGAGAACATTTCCC...,TTCAGaaaaaaaaattaatttattttaCCAGCAGTATTCATAGCTG...
2,Fzd4,chr7,+,89404365,89410110,89404686,89408360,89407031,89410110,3079,...,89407633,89407634,0.908575,1,AGCGCTGGGGCGGTGAGAACAGCGCGGCGTAGAGTGCAGGCGGGCT...,5745,3268,AGCGCTGGGGCGGTGAGAACAGCGCGGCGTAGAGTGCAGGCGGGCT...,GCTCAGCTAAGGAGTTCACGGATATTTGGATGGCTGTGTGGGCCAG...,GGAACCAAAATCTAGATGCCCTCACTGGCTTTGTGGTGGCTCCTCT...
3,Pard6b,chr2,+,168081003,168101203,168081293,168099210,168098383,168101203,2820,...,168099166,168099167,0.906320,1,AGGGAGGCTGCGGCGGCTGTAGGGCTGTGCTCCGCCCTCTCCTCAc...,20200,18163,AGGGAGGCTGCGGCGGCTGTAGGGCTGTGCTCCGCCCTCTCCTCAc...,GGCTTCCCACAGCAGGTGGAGGCCAGCTTCGAGCCCGAGGACCAGG...,TCCACCTGCCTGCAGAGCCCGCTCCTTGCCCTCTGTGAAGTCTGAT...
4,Rif1,chr2,+,52072836,52122381,52073149,52121203,52109910,52113000,3090,...,52112643,52112644,0.898967,1,ATCTTGGTCGTGGAGGAGCGGGCTGCACGCGTGAGTAAATAAGCGC...,49545,39807,ATCTTGGTCGTGGAGGAGCGGGCTGCACGCGTGAGTAAATAAGCGC...,TGAGACTGAGCACGCAGCTAGTGGGGAAATAGAGGGGGAGAGCAAT...,GCAGCAGGCACGTTGTGTCTGGTCACCTTTGGCTTCTCCATCCACA...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
888,Slc22a23,chr13,-,34179157,34345182,34182953,34344797,34179157,34183311,4154,...,34183035,34183036,0.405162,1,GAAGTCCGTGGAGGATCTCCCAAcccgcggccgccggcggccccAG...,166025,162146,GAAGTCCGTGGAGGATCTCCCAAcccgcggccgccggcggccccAG...,GCCAGTGCAGGTTTTGGCATGCTGACAGCGCCCATTATTGAGCTGC...,GGAACACATCTGAGAAACCAGATGCTCTGGAAATAACCCTCTGAAG...
889,Eri2,chr7,-,119784895,119794058,119785209,119793940,119784895,119786544,1649,...,119785365,119785366,0.404177,1,GACGGCCGCCGCCGCTCTACAAGCGCCCGCTTCCCTTGCCGCGCCG...,9163,8692,GACGGCCGCCGCCGCTCTACAAGCGCCCGCTTCCCTTGCCGCGCCG...,AGCCTACATCATCTATTTGCTCCCCAGGAACAACTTCTTGTAGAGT...,tctgagcactgcagctacaccggagcagttcatacaaacagaaggg...
890,Wtap,chr17,-,12966798,12992259,12967467,12985945,12966798,12968051,1253,...,12967501,12967502,0.403611,1,agtccgggcggggcggggccggcggcagggccggcTTTCCTCCCTC...,25461,24757,agtccgggcggggcggggccggcggcagggccggcTTTCCTCCCTC...,TTTCCTTCTTCTTCAGGGAATGGTAATAAGGCCTCCAACAGCTCAG...,ATTGCTAAAAGGACATTTTGTGTAGGGTCAAGTTATTTTTATATGA...
891,Lrrn3,chr12,-,41451672,41485751,41452192,41454316,41451672,41454671,2999,...,41452631,41452632,0.403062,1,agagagaCACCATTGCTTGCATGTGTGCATATCTATAGTATGTAAT...,34079,33119,agagagaCACCATTGCTTGCATGTGTGCATATCTATAGTATGTAAT...,GGCACACTAGAGATAAGAGGCATAACCCCAAAGGAAGGGGGGTTGT...,AGCTGTGTCTCACAAGAAGGGAGCAGTGAGGGTGAGCACAGCTATG...


In [34]:
Data.to_csv("Fasta_SNM_LongInternalExon.csv", index=0)